In [1]:
import numpy as np
import scipy
import pandas
import torch
import utils
from utils import generate_state_space_Matern_23
from scipy import linalg


In [2]:
data_file = '../processed_data/beijing_15k.npy'
data_dict = np.load(data_file, allow_pickle=True).item()

fold=0

# here should add one more data-loader

hyper_dict={}
hyper_dict['ls'] = 0.5
hyper_dict['var'] = 0.1
hyper_dict['device'] = torch.device("cpu")
hyper_dict['R_U'] = 5 # dim of each node embedding

F,P_inf = utils.generate_state_space_Matern_23(data_dict,hyper_dict,fold)

In [ ]:
# data process:

class Bayes_diff_tensor():
    def __init__(self,data_dict,hyper_dict):

        # training data
        self.ind_tr = data_dict['ind_tr']
        self.y_tr = data_dict['y_tr'].to(self.device) # N*1
        
        self.train_time_ind =data_dict['train_time_ind'] 
        self.test_time_ind = data_dict['test_time_ind']   
        self.N_time = data_dict['N_time'] 
        self.time_data_table = data_dict['time_data_table']

In [3]:
F,P_inf = utils.generate_state_space_Matern_23(data_dict,hyper_dict,fold)



tensor([[ 6.1316e-03,  3.1644e-03,  3.1488e-03,  ..., -1.0340e-03,
         -9.2864e-04, -9.7887e-04],
        [ 3.1644e-03,  6.4374e-03,  3.1514e-03,  ..., -1.0340e-03,
         -9.2855e-04, -9.7886e-04],
        [ 3.1488e-03,  3.1514e-03,  6.8081e-03,  ..., -9.8918e-04,
         -8.7936e-04, -9.3269e-04],
        ...,
        [-1.0340e-03, -1.0340e-03, -9.8918e-04,  ...,  1.1276e+00,
          3.8169e-03,  3.7441e-03],
        [-9.2864e-04, -9.2855e-04, -8.7936e-04,  ...,  3.8169e-03,
          1.1220e+00,  3.7852e-03],
        [-9.7887e-04, -9.7886e-04, -9.3269e-04,  ...,  3.7441e-03,
          3.7852e-03,  1.1249e+00]], dtype=torch.float64)

In [10]:
for item in list(np.unique(data_dict['t_te'])):
    if item not in list(np.unique(data_dict['t'])):
        print('no')

In [9]:
0.0 in list(np.unique(data_dict['t']))

True